Make raw data for testing

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

class CustomFCNN(nn.Module):
    def __init__(self, depth, units, input_size=40, output_size=10):
        super(CustomFCNN, self).__init__()
        self.depth = depth
        self.units = units
        self.input_size = input_size
        self.output_size = output_size
        layers = []
        for i in range(depth):
            layers.append(nn.Linear(units[i], units[i + 1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(units[-2], units[-1]))
        self.layers = nn.Sequential(*layers)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        return self.layers(x)

    def get_total_params(self):
        return get_total_params_function(self.depth, self.units)

def get_total_params_function(depth, units):
    total_params = 0
    for layer in range(1, depth + 2):
        params = units[layer] * (units[layer - 1] + 1)
        total_params += params
    return total_params

def get_units_equal(input_size, output_size, depth, params_limit=15500):
    units = [input_size]
    possible_num_of_unit = 0
    test = 1
    while True:
        test_units = [input_size] + [test] * depth + [output_size]
        test_params = get_total_params_function(depth, test_units)
        if test_params > params_limit:
            break
        else:
            possible_num_of_unit = test
            test += 1
    units.extend([possible_num_of_unit] * depth)
    units.append(output_size)
    return units


In [13]:
import pandas as pd

column_names = ['depth', 'step_C', 'loss*', 'loss_t0', 'diff_loss', 'nor_diff_loss', 'L2', 'nor_L2', 'cos_sim', 'CR', 'nor_CR']
df_total = pd.DataFrame(columns=column_names)

In [14]:
import numpy as np

def L2(vec1, vec2):
    # ||w* - w0||2
    return np.linalg.norm(vec2 - vec1)

def nor_L2(vec1, vec2):
    # ||w* - w0||2 / ||w0||
    return L2(vec1, vec2) / np.linalg.norm(vec1)

def cos_sim(vec1, vec2):
  return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

In [15]:
for depth in range(1, 8):
    # CSV file path
    RESULT_PATH = f"./result/df/result_depth_{depth}.csv"

    # load data frame
    df = pd.read_csv(RESULT_PATH)
    df['depth'] = depth

    # Calculate with weight
    units = get_units_equal(40, 10, depth, 15000 + 500)
    model = CustomFCNN(depth=depth, units=units)

    model_t0_path = f'/home/work/DL_term_project/result/model/w_t0_depth_{depth}.pth'
    model.load_state_dict(torch.load(model_t0_path))
    model.eval()
    state_dict = model.state_dict()
    w0 = torch.cat([param.view(-1) for param in state_dict.values()]).numpy()

    model_path = f'/home/work/DL_term_project/result/model/w*_depth_{depth}.pth'
    model.load_state_dict(torch.load(model_path))
    model.eval()
    state_dict = model.state_dict()
    w = torch.cat([param.view(-1) for param in state_dict.values()]).numpy()

    df['L2'] = L2(w0, w)
    df['nor_L2'] = nor_L2(w0, w)
    df['cos_sim'] = cos_sim(w0, w)
    
    # Save
    df_total = pd.concat([df_total, df], ignore_index=True)


df_total

,depth,step_C,loss*,loss_t0,diff_loss,nor_diff_loss,L2,nor_L2,cos_sim,CR,nor_CR
0,1,360280,0.0712,3.0181,NaN,NaN,26.686325,1.062529,0.745458,NaN,NaN
1,2,140458,0.0534,3.3110,NaN,NaN,17.400347,0.841367,0.821232,NaN,NaN
2,3,103553,0.0394,3.4110,NaN,NaN,14.853483,0.679567,0.872377,NaN,NaN
3,4,73262,0.0469,2.7387,NaN,NaN,12.469867,0.541258,0.908195,NaN,NaN
4,5,63049,0.0308,3.2568,NaN,NaN,11.425949,0.475164,0.926300,NaN,NaN
5,6,57576,0.0293,3.4151,NaN,NaN,10.686932,0.424923,0.939397,NaN,NaN
6,7,54115,0.0232,3.2331,NaN,NaN,10.191139,0.391578,0.946856,NaN,NaN


In [16]:
# Calculate other columns
df_total['diff_loss'] = df_total['loss*'] - df_total['loss_t0']
df_total['nor_diff_loss'] = (df_total['loss*'] - df_total['loss_t0']) / df_total['loss_t0']
df_total['CR'] = df_total['L2'] / df_total['step_C']
df_total['nor_CR'] = df_total['nor_L2'] / df_total['step_C']

df_total

,depth,step_C,loss*,loss_t0,diff_loss,nor_diff_loss,L2,nor_L2,cos_sim,CR,nor_CR
0,1,360280,0.0712,3.0181,-2.9469,-0.976409,26.686325,1.062529,0.745458,0.000074,0.000003
1,2,140458,0.0534,3.3110,-3.2576,-0.983872,17.400347,0.841367,0.821232,0.000124,0.000006
2,3,103553,0.0394,3.4110,-3.3716,-0.988449,14.853483,0.679567,0.872377,0.000143,0.000007
3,4,73262,0.0469,2.7387,-2.6918,-0.982875,12.469867,0.541258,0.908195,0.00017,0.000007
4,5,63049,0.0308,3.2568,-3.2260,-0.990543,11.425949,0.475164,0.926300,0.000181,0.000008
5,6,57576,0.0293,3.4151,-3.3858,-0.991420,10.686932,0.424923,0.939397,0.000186,0.000007
6,7,54115,0.0232,3.2331,-3.2099,-0.992824,10.191139,0.391578,0.946856,0.000188,0.000007


In [17]:
df_total['CR_scale'] = df_total['L2'] / df_total['step_C'] * 10000
df_total['nor_CR_scale'] = df_total['nor_L2'] / df_total['step_C'] * 10000

df_total

,depth,step_C,loss*,loss_t0,diff_loss,nor_diff_loss,L2,nor_L2,cos_sim,CR,nor_CR,CR_scale,nor_CR_scale
0,1,360280,0.0712,3.0181,-2.9469,-0.976409,26.686325,1.062529,0.745458,0.000074,0.000003,0.740711,0.029492
1,2,140458,0.0534,3.3110,-3.2576,-0.983872,17.400347,0.841367,0.821232,0.000124,0.000006,1.238829,0.059902
2,3,103553,0.0394,3.4110,-3.3716,-0.988449,14.853483,0.679567,0.872377,0.000143,0.000007,1.434385,0.065625
3,4,73262,0.0469,2.7387,-2.6918,-0.982875,12.469867,0.541258,0.908195,0.00017,0.000007,1.702092,0.07388
4,5,63049,0.0308,3.2568,-3.2260,-0.990543,11.425949,0.475164,0.926300,0.000181,0.000008,1.812233,0.075364
5,6,57576,0.0293,3.4151,-3.3858,-0.991420,10.686932,0.424923,0.939397,0.000186,0.000007,1.856143,0.073802
6,7,54115,0.0232,3.2331,-3.2099,-0.992824,10.191139,0.391578,0.946856,0.000188,0.000007,1.883237,0.07236


In [18]:
df_total.to_csv('/home/work/DL_term_project/result/df/result_total.csv', index=False)

Correlation Test

In [19]:
import pandas as pd
import scipy.stats as stats

df_total = pd.read_csv('/home/work/DL_term_project/result/df/result_total.csv',)
df_total['diff_loss'] = df_total['diff_loss'].abs()
df_total['nor_diff_loss'] = df_total['nor_diff_loss'].abs()
df_total['cos_dis']= 1 - df_total['cos_sim']
df_total = df_total[['depth', 'diff_loss', 'nor_diff_loss', 'cos_dis', 'L2', 'nor_L2']]

df_total

,depth,diff_loss,nor_diff_loss,cos_dis,L2,nor_L2
0,1,2.9469,0.976409,0.254542,26.686325,1.062529
1,2,3.2576,0.983872,0.178767,17.400347,0.841368
2,3,3.3716,0.988449,0.127623,14.853483,0.679567
3,4,2.6918,0.982875,0.091805,12.469867,0.541257
4,5,3.2260,0.990543,0.073700,11.425949,0.475164
5,6,3.3858,0.991420,0.060603,10.686932,0.424923
6,7,3.2099,0.992824,0.053144,10.191139,0.391578


In [20]:
# Create an empty DataFrame for the new rows
new_rows_df = pd.DataFrame(columns=df_total.columns)
new_row = {col: "." for col in df_total.columns}

# Set alpha 
alpha = 0.05

# Conduct Spearman rank correaltion test between depth and other columns
columns_to_test = ['diff_loss', 'nor_diff_loss', 'cos_dis', 'L2', 'nor_L2']

results = {}
for col in columns_to_test:
    corr, p_value = stats.spearmanr(df_total['depth'], df_total[col])

    # Mark significance stars based on p-value
    if p_value < 0.001:
        significance = '**'
    elif p_value < 0.01:
        significance = '*'
    elif p_value < 0.05:
        significance = ''
    else:
        significance = ''

    results[col] = {'Spearman Correlation': corr, 'p-value': p_value, 'significance': significance}


# Result
for col, result in results.items():
    print(f"{col}: Spearman Correlation = {result['Spearman Correlation']:.4f}, p-value = {result['p-value']:.4e} {result['significance']}")
    new_row[col] = f"{result['Spearman Correlation']:.4f} (p={result['p-value']:.4e}) {result['significance']}"

# Add the new row for Spearman Correlation
new_row_spearman = new_row
new_rows_df = new_rows_df.append(new_row_spearman, ignore_index=True)

diff_loss: Spearman Correlation = 0.1786, p-value = 7.0166e-01 
nor_diff_loss: Spearman Correlation = 0.8929, p-value = 6.8072e-03 *
cos_dis: Spearman Correlation = -1.0000, p-value = 0.0000e+00 **
L2: Spearman Correlation = -1.0000, p-value = 0.0000e+00 **
nor_L2: Spearman Correlation = -1.0000, p-value = 0.0000e+00 **


/tmp/ipykernel_73005/2376266438.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows_df = new_rows_df.append(new_row_spearman, ignore_index=True)


In [21]:
new_row = {col: "." for col in df_total.columns}

# Set alpha
alpha = 0.05

# Conduct Spearman rank correaltion test between depth and other columns
columns_to_test = ['diff_loss', 'nor_diff_loss', 'cos_dis', 'L2', 'nor_L2']

results = {}
for col in columns_to_test:
    if col != 'depth':
        corr, p_value = stats.pearsonr(df_total['depth'], df_total[col])
        results[col] = {'Pearson Correlation': corr, 'p-value': p_value}

    # Mark significance stars based on p-value
    if p_value < 0.001:
        significance = '**'
    elif p_value < 0.01:
        significance = '*'
    elif p_value < 0.05:
        significance = ''
    else:
        significance = ''

    results[col] = {'Pearson Correlation': corr, 'p-value': p_value, 'significance': significance}

# Result
for col, result in results.items():
    print(f"{col}: Pearson Correlation = {result['Pearson Correlation']:.4f}, p-value = {result['p-value']:.4e} {result['significance']}")
    new_row[col] = f"{result['Pearson Correlation']:.4f} (p={result['p-value']:.4e}) {result['significance']}"

# Add the new row for Pearson Correlation
new_row_pearson = new_row
new_rows_df = new_rows_df.append(new_row_pearson, ignore_index=True)

diff_loss: Pearson Correlation = 0.2771, p-value = 5.4745e-01 
nor_diff_loss: Pearson Correlation = 0.8745, p-value = 9.9988e-03 *
cos_dis: Pearson Correlation = -0.9369, p-value = 1.8551e-03 *
L2: Pearson Correlation = -0.8799, p-value = 8.9930e-03 *
nor_L2: Pearson Correlation = -0.9542, p-value = 8.4319e-04 **


/tmp/ipykernel_73005/1831394420.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_rows_df = new_rows_df.append(new_row_pearson, ignore_index=True)


In [22]:
# Append the new rows to the original DataFrame
df_total = pd.concat([df_total, new_rows_df], ignore_index=True)
df_total.index = df_total.index+1
df_total = df_total[['L2', 'nor_L2', 'cos_dis', 'diff_loss', 'nor_diff_loss']]

df_total_T = df_total.transpose()
df_total_T

,1,2,3,4,5,6,7,8,9
L2,26.686325,17.400347,14.853483,12.469867,11.425949,10.686932,10.191139,-1.0000 (p=0.0000e+00) **,-0.8799 (p=8.9930e-03) *
nor_L2,1.062529,0.841368,0.679567,0.541257,0.475164,0.424923,0.391578,-1.0000 (p=0.0000e+00) **,-0.9542 (p=8.4319e-04) **
cos_dis,0.254542,0.178767,0.127623,0.091805,0.0737,0.060603,0.053144,-1.0000 (p=0.0000e+00) **,-0.9369 (p=1.8551e-03) *
diff_loss,2.9469,3.2576,3.3716,2.6918,3.226,3.3858,3.2099,0.1786 (p=7.0166e-01),0.2771 (p=5.4745e-01)
nor_diff_loss,0.976409,0.983872,0.988449,0.982875,0.990543,0.99142,0.992824,0.8929 (p=6.8072e-03) *,0.8745 (p=9.9988e-03) *
